In [7]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
import numpy as np


In [9]:
# client = AlpacaClient()
window = 100
extractor = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BCH", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
tickers = [x +"/USD" for x in tickers]
prices = []
start = datetime.now() - timedelta(days=365)
end = datetime.now()
for ticker in tickers:
    response = extractor.crypto(ticker,start,end)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c":"adjclose","t":"date"})[["date","adjclose"]]
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    price["rolling_return"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"]) / price["adjclose"])
    price["bollinger"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"].rolling(window).std() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(5)
    price["avg_return"] = price["return"].rolling(window).mean()
    price["coev"] = (price["adjclose"].rolling(window).mean() / price["adjclose"].rolling(window).std())
    price["ticker"]= ticker
    prices.append(price)

sim = pd.concat(prices).sort_values("date").dropna()

In [10]:
metrics = [
    "rolling_return",
    "bollinger",
    "return",
    "avg_return",
    "coev"
]

In [14]:
import copy
positions = []
analysis = []
cash = 100
date_range = list(sim["date"].unique())
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
for metric in metrics:
    for ascending in [True,False]:
        position = copy.deepcopy(base_position)
        for date in tqdm(date_range[:1000]):
            try:
                position = copy.deepcopy(position)
                position["date"] = date
                today = sim[sim["date"]==date]
                if today.index.size >= 0:
                    opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        if (position["adjclose"] - position["buy_price"]) / position["buy_price"] > 0.05:
                            try:
                                cash = position["adjclose"] * position["quantity"]
                                position = copy.deepcopy(base_position)
                                position["date"] = date
                                position["ticker"] = opportunity["ticker"]
                                position["quantity"] = cash * 0.996 / opportunity["adjclose"] 
                                position["adjclose"] = opportunity["adjclose"]
                                position["buy_price"] = opportunity["adjclose"]
                                position["buy_date"] = date
                            except Exception as e:
                                print(str(e))
                                continue
                    else:
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.996 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
            except Exception as e:
                print(str(e))
        position["metric"] = metric
        position["ascending"] = ascending
        position["window"] = window
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(copy.deepcopy(position))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262/262 [00:00<00:00, 1253.31it/s]


In [15]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,window,pv
9,2024-09-26 05:00:00+00:00,YFI/USD,1.912367e+00,5285.548500,9049.356500,2024-03-29 05:00:00+00:00,coev,False,100,10107.906943
8,2024-09-26 05:00:00+00:00,SHIB/USD,2.931299e+08,0.000019,0.000036,2024-03-08 06:00:00+00:00,coev,True,100,5443.421878
7,2024-09-26 05:00:00+00:00,SHIB/USD,1.971607e+08,0.000019,0.000036,2024-03-05 06:00:00+00:00,avg_return,False,100,3661.274671
6,2024-09-26 05:00:00+00:00,YFI/USD,4.458740e-01,5285.548500,10100.930000,2024-03-04 06:00:00+00:00,avg_return,True,100,2356.688863
5,2024-09-26 05:00:00+00:00,SHIB/USD,8.839620e+07,0.000019,0.000039,2024-03-04 06:00:00+00:00,return,False,100,1641.517489
4,2024-09-26 05:00:00+00:00,YFI/USD,1.413460e-01,5285.548500,9273.390000,2024-03-25 05:00:00+00:00,return,True,100,747.091340
3,2024-09-26 05:00:00+00:00,LINK/USD,3.944578e+01,12.525650,20.732750,2024-03-11 05:00:00+00:00,bollinger,False,100,494.084024
2,2024-09-26 05:00:00+00:00,SHIB/USD,1.297372e+07,0.000019,0.000039,2024-03-04 06:00:00+00:00,bollinger,True,100,240.921905
1,2024-09-26 05:00:00+00:00,LINK/USD,9.961482e+00,12.525650,20.732750,2024-03-11 05:00:00+00:00,rolling_return,False,100,124.774041
0,2024-09-26 05:00:00+00:00,SHIB/USD,3.753301e+06,0.000019,0.000032,2024-03-10 06:00:00+00:00,rolling_return,True,100,69.698806
